In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
import time

In [4]:

# Leer el archivo de Spotify (ajusta el path si es necesario)
df_artists = pd.read_csv("../data/artists.csv")

artistas_raw = df_artists['artist'].dropna().unique()
artistas = set()

# Función para separar artistas con múltiples delimitadores
def split_artists(texto):
    # Reemplazar "Featuring" por un delimitador temporal (e.g., ";") y limpiar
    texto = re.sub(r'\s*Featuring\s*', ',', texto, flags=re.IGNORECASE)
    # Separar por ";" y "&"
    partes = re.split(r'[,&]', texto)
    # Limpiar cada artista y devolver lista
    return [artista.strip() for artista in partes if artista.strip()]

# Procesar cada grupo de artistas
for grupo in artistas_raw:
    for artista in split_artists(grupo):
        artistas.add(artista)

print(f"🎵 Total de artistas únicos: {len(artistas)}")

🎵 Total de artistas únicos: 2967


In [5]:
import pandas as pd
import requests
import time
from tqdm import tqdm

# Cargar archivo original
df = pd.read_csv("../data/artists.csv", header=None, names=["artist_raw"])

# Limpieza básica
def limpiar_nombre(artista):
    if pd.isna(artista):
        return None
    artista = artista.replace('"', '')
    artista = artista.split(" Featuring ")[0]
    artista = artista.split(" & ")[0]
    artista = artista.split(",")[0]
    return artista.strip()

df["artist"] = df["artist_raw"].apply(limpiar_nombre)

# Eliminar duplicados y valores vacíos
unique_artists = df["artist"].dropna().drop_duplicates().reset_index(drop=True)

# API de TheAudioDB (requiere APIKEY)
API_KEY = "2"  # Puedes usar '1' para acceso limitado sin registro
BASE_URL = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/search.php?s="

# Lista para almacenar resultados
results = []

# Iterar sobre los artistas
for artist in tqdm(unique_artists, desc="Procesando artistas"):
    url = BASE_URL + requests.utils.quote(artist)
    for intento in range(3):
        try:
            response = requests.get(url, timeout=10)
            data = response.json()
            if data and data.get("artists"):
                artist_data = data["artists"][0]
                results.append({
                    "artist_name": artist,
                    "idArtist": artist_data.get("idArtist"),
                    "strCountry": artist_data.get("strCountry"),
                    "intFormedYear": artist_data.get("intFormedYear"),
                    "intDiedYear": artist_data.get("intDiedYear"),
                    "strArtistType": artist_data.get("strArtistType"),
                    "strGender": artist_data.get("strGender"),
                    "strBiographyEN": artist_data.get("strBiographyEN")
                })
                break
            else:
                print(f"⚠️ Respuesta vacía para {artist}. Intento {intento + 1}/3")
        except Exception as e:
            print(f"Excepción para {artist}: {e}. Intento {intento + 1}/3")
        time.sleep(1)

# Convertir a DataFrame y guardar
df_results = pd.DataFrame(results)
df_results.to_csv("../data/api_data.csv", index=False)
print("✅ Extracción finalizada. Datos guardados en '../data/api_data.csv'")


Procesando artistas:   0%|          | 0/2110 [00:00<?, ?it/s]

Excepción para artist: Expecting value: line 1 column 1 (char 0). Intento 1/3
Excepción para artist: Expecting value: line 1 column 1 (char 0). Intento 2/3
Excepción para artist: Expecting value: line 1 column 1 (char 0). Intento 3/3


Procesando artistas:   0%|          | 1/2110 [00:15<8:59:41, 15.35s/it]

Excepción para Billie Eilish: Expecting value: line 1 column 1 (char 0). Intento 1/3
Excepción para Billie Eilish: Expecting value: line 1 column 1 (char 0). Intento 2/3
Excepción para Billie Eilish: Expecting value: line 1 column 1 (char 0). Intento 3/3


Procesando artistas:   0%|          | 2/2110 [00:19<5:08:02,  8.77s/it]

Excepción para Bon Iver: Expecting value: line 1 column 1 (char 0). Intento 1/3
Excepción para Bon Iver: Expecting value: line 1 column 1 (char 0). Intento 2/3
Excepción para Bon Iver: Expecting value: line 1 column 1 (char 0). Intento 3/3


Procesando artistas:   0%|          | 3/2110 [00:23<3:53:41,  6.65s/it]

Excepción para Ariana Grande: Expecting value: line 1 column 1 (char 0). Intento 1/3


Procesando artistas:   0%|          | 3/2110 [00:25<4:53:27,  8.36s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import time

# Limpieza básica del nombre del artista
def limpiar_nombre(artista):
    if pd.isna(artista):
        return None
    artista = artista.replace('"', '')
    artista = artista.split(" Featuring ")[0]
    artista = artista.split(" & ")[0]
    artista = artista.split(",")[0]
    return artista.strip()

# Función para obtener el MBID desde MusicBrainz
def obtener_mbid(nombre):
    url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{nombre}&fmt=json"
    try:
        r = requests.get(url, headers={"User-Agent": "ArtistETL/1.0 (example@email.com)"})
        data = r.json()
        for artista in data.get("artists", []):
            if artista["name"].lower() == nombre.lower():
                return artista["id"]
    except Exception as e:
        print(f"Error MBID para {nombre}: {e}")
    return None

# Función para consultar Wikidata con el MBID
def consultar_wikidata(mbid):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = f"""
    SELECT ?itemLabel ?countryLabel ?genderLabel ?typeLabel ?start ?end WHERE {{
      ?item wdt:P434 "{mbid}".
      OPTIONAL {{ ?item wdt:P27 ?country. }}
      OPTIONAL {{ ?item wdt:P21 ?gender. }}
      OPTIONAL {{ ?item wdt:P31 ?type. }}
      OPTIONAL {{ ?item wdt:P2031 ?start. }}
      OPTIONAL {{ ?item wdt:P2032 ?end. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]
        if bindings:
            res = bindings[0]
            return {
                "pais": res.get("countryLabel", {}).get("value", None),
                "genero": res.get("genderLabel", {}).get("value", None),
                "tipo": res.get("typeLabel", {}).get("value", None),
                "activo_desde": res.get("start", {}).get("value", None),
                "activo_hasta": res.get("end", {}).get("value", None)
            }
    except Exception as e:
        print(f"Error Wikidata para {mbid}: {e}")
    return {}

# Cargar artistas y limpiar nombres
df = pd.read_csv("../data/artists.csv", header=None, names=["artist_raw"])
df["artist"] = df["artist_raw"].apply(limpiar_nombre)
unique_artists = df["artist"].dropna().drop_duplicates().reset_index(drop=True)

# Procesar artistas
resultados = []
for nombre in tqdm(unique_artists, desc="Procesando artistas"):
    mbid = obtener_mbid(nombre)
    if mbid:
        info = consultar_wikidata(mbid)
        info["nombre"] = nombre
        info["mbid"] = mbid
        resultados.append(info)
    time.sleep(1.1)  # Para evitar abusar del endpoint

# Guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("artistas_info.csv", index=False)
print("✅ Datos guardados en artistas_info.csv")


In [6]:
import pandas as pd
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import math

HEADERS = {"User-Agent": "ArtistETL/1.0 (your_email@example.com)"}
SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"

# ------------------------------
# Limpieza de nombre
def limpiar_nombre(artista):
    if pd.isna(artista): return None
    artista = artista.replace('"', '')
    artista = artista.split(" Featuring ")[0]
    artista = artista.split(" & ")[0]
    artista = artista.split(",")[0]
    return artista.strip()

# ------------------------------
# Obtener MBID desde MusicBrainz
def obtener_mbid(nombre):
    url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{nombre}&fmt=json"
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        data = r.json()
        for artista in data.get("artists", []):
            if artista["name"].lower() == nombre.lower():
                return (nombre, artista["id"])
    except:
        return (nombre, None)
    return (nombre, None)

# ------------------------------
# Crear bloques de MBIDs
def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

# ------------------------------
# Consultar Wikidata por lote
def consultar_lote_wikidata(mbids):
    filtro = " ".join([f'"{mbid}"' for mbid in mbids])
    query = f"""
    SELECT ?mbid ?itemLabel ?countryLabel ?genderLabel ?typeLabel ?start ?end WHERE {{
      VALUES ?mbid_str {{ {filtro} }}
      ?item wdt:P434 ?mbid_str.
      BIND(?mbid_str AS ?mbid)
      OPTIONAL {{ ?item wdt:P27 ?country. }}
      OPTIONAL {{ ?item wdt:P21 ?gender. }}
      OPTIONAL {{ ?item wdt:P31 ?type. }}
      OPTIONAL {{ ?item wdt:P2031 ?start. }}
      OPTIONAL {{ ?item wdt:P2032 ?end. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        return results["results"]["bindings"]
    except Exception as e:
        print("❌ Error en SPARQL:", e)
        return []

# ------------------------------
# Paso 1: Cargar artistas
df = pd.read_csv("../data/artists.csv", header=None, names=["artist_raw"])
df["artist"] = df["artist_raw"].apply(limpiar_nombre)
unique_artists = df["artist"].dropna().drop_duplicates().reset_index(drop=True)

# Paso 2: Obtener MBIDs en paralelo
print("🔍 Buscando MBIDs...")
mbid_map = {}
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(obtener_mbid, name) for name in unique_artists]
    for f in tqdm(as_completed(futures), total=len(futures)):
        name, mbid = f.result()
        if mbid:
            mbid_map[name] = mbid

# Paso 3: Consultar Wikidata por bloques
print("🌐 Consultando Wikidata en bloques...")
todos_resultados = []

batch_size = 40  # Puedes ajustar a 50 o más si quieres
bloques = list(chunk_list(list(mbid_map.values()), batch_size))

for bloque in tqdm(bloques, desc="Batches SPARQL"):
    resultados = consultar_lote_wikidata(bloque)
    for r in resultados:
        todos_resultados.append({
            "mbid": r.get("mbid", {}).get("value"),
            "nombre": r.get("itemLabel", {}).get("value"),
            "pais": r.get("countryLabel", {}).get("value"),
            "genero": r.get("genderLabel", {}).get("value"),
            "tipo": r.get("typeLabel", {}).get("value"),
            "activo_desde": r.get("start", {}).get("value"),
            "activo_hasta": r.get("end", {}).get("value"),
        })
    time.sleep(1.1)  # Respetamos límites de Wikidata (1s por consulta)

# Paso 4: Guardar resultados
pd.DataFrame(todos_resultados).to_csv("wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")


🔍 Buscando MBIDs...


100%|██████████| 2110/2110 [01:52<00:00, 18.74it/s]


🌐 Consultando Wikidata en bloques...


Batches SPARQL:  75%|███████▌  | 3/4 [00:05<00:01,  1.86s/it]

❌ Error en SPARQL: HTTP Error 403: Forbidden


Batches SPARQL: 100%|██████████| 4/4 [00:07<00:00,  1.80s/it]

✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [13]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# =====================
# 1. CARGAR Y LIMPIAR
# =====================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    artistas_limpios = [p.strip().strip('"').strip("'") for p in partes if p.strip()]
    return artistas_limpios

# Expandir y limpiar todos los artistas
todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

# Quitar duplicados
artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))

print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =====================
# 2. FUNCIONES SPARQL
# =====================

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel WHERE {{
      VALUES ?artistLabel {{ {values} }}
      
      ?artist rdfs:label ?artistLabel.
      FILTER(LANG(?artistLabel) = "en")
      
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.get(WIKIDATA_ENDPOINT, params={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =====================
# 3. PROCESAR BATCHES
# =====================
print("🚀 Consultando Wikidata...")

batch_size = 50
results = []

for i in tqdm(range(0, len(artistas_unicos), batch_size), desc="🔎 Batches SPARQL"):
    batch = artistas_unicos[i:i+batch_size]
    data = obtener_datos_wikidata(batch)
    if data:
        for row in data["results"]["bindings"]:
            results.append({
                "artist": row.get("artistLabel", {}).get("value", ""),
                "country": row.get("countryLabel", {}).get("value", ""),
                "type": row.get("typeLabel", {}).get("value", ""),
                "genre": row.get("genreLabel", {}).get("value", ""),
                "birth": row.get("birth", {}).get("value", ""),
                "death": row.get("death", {}).get("value", "")
            })
    time.sleep(1.2)  # respetar política de uso

# =====================
# 4. GUARDAR RESULTADO
# =====================
df_result = pd.DataFrame(results)

# Inferir si está activo o no
df_result["is_active"] = df_result["death"].apply(lambda x: "No" if x else "Yes")

df_result.to_csv("wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")


🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30172
🚀 Consultando Wikidata...


🔎 Batches SPARQL:   6%|▌         | 35/604 [01:03<45:33,  4.80s/it]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%0A++++SELECT+%3FartistLabel+%3Fbirth+%3Fdeath+%3FcountryLabel+%3FtypeLabel+%3FgenreLabel+WHERE+%7B%0A++++++VALUES+%3FartistLabel+%7B+%22Anthony+Rother%22%0A%22Anthony+Russo%22%0A%22Anthony+Santos%22%0A%22Anthony+Shakir%22%0A%22Anthony+Weeden%22%0A%22Anthony+Wong%22%0A%22Anthrax%22%0A%22Anthropological%22%0A%22Anti%22%0A%22Antibalas%22%0A%22Antichrist+Siege+Machine%22%0A%22Antidoping%22%0A%22Antifuchs%22%0A%22Antigama%22%0A%22Antillas%22%0A%22Antinomy%22%0A%22Anton+Ishutin%22%0A%22Anton+Mittermayr%22%0A%22Anton+Powers%22%0A%22Anton+Steck%22%0A%22Antonello+Leofreddi%22%0A%22Antonia+Bennett%22%0A%22Antonia+Fahberg%22%0A%22Antonino+Votto%22%0A%22Antonio+Agri%22%0A%22Antonio+Biribiesca%22%0A%22Antonio+Bribiesca%22%0A%22Antonio+Caldara%22%0A%22Antonio+Carlos%22%0A%22Antonio+Carmona%22%0A%22Antonio+Cartagena%22%0A%22Antonio+Cirilo%22%0A%22Antonio+Garcia+Isaac%22%0A%22Antonio+Jes%C3%BAs+Asi%C3%A1

🔎 Batches SPARQL:  11%|█▏        | 68/604 [01:52<13:34,  1.52s/it]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%0A++++SELECT+%3FartistLabel+%3Fbirth+%3Fdeath+%3FcountryLabel+%3FtypeLabel+%3FgenreLabel+WHERE+%7B%0A++++++VALUES+%3FartistLabel+%7B+%22Blue+States%22%0A%22Blue+Swede%22%0A%22Blue+%C3%96yster+Cult%22%0A%22Blueboy%22%0A%22Blueridge+Mountain+Bluegrass+Band%22%0A%22Blues+Delight%22%0A%22Blues+Saraceno%22%0A%22BluntOne%22%0A%22Blunts%22%0A%22Blur%22%0A%22Blurstem%22%0A%22Blush%22%0A%22Bluszcz%22%0A%22Blutengel%22%0A%22Blvk+Sheep%22%0A%22Blxst%22%0A%22Bl%C3%A5+T%C3%A5get%22%0A%22Bl%C3%BCmchen%22%0A%22Bo+Bruce%22%0A%22Bo+Diddley%22%0A%22Bo+Kaspers+Orkester%22%0A%22Bo+Sundstr%C3%B6m%22%0A%22Boards+of+Canada%22%0A%22Boaz+Daniel%22%0A%22Boaz+van+de+Beatz%22%0A%22Bob%22%0A%22Bob+Dylan%22%0A%22Bob+Gunton%22%0A%22Bob+Luman%22%0A%22Bob+Marley%22%0A%22Bob+Moses%22%0A%22Bob+Mould%22%0A%22Bob+Saget%22%0A%22Bob+Seger%22%0A%22Bob+Sinclar%22%0A%22Bob+Smith%22%0A%22Bob+Stevens%22%0A%22Bob+Weir%22%0A%22Bobbi+

...

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%0A++++SELECT+%3FartistLabel+%3Fbirth+%3Fdeath+%3FcountryLabel+%3FtypeLabel+%3FgenreLabel+WHERE+%7B%0A++++++VALUES+%3FartistLabel+%7B+%22Coldcut%22%0A%22Colde%22%0A%22Coldplay%22%0A%22Coldworker%22%0A%22Cole+Sprouse%22%0A%22Cole+Swindell%22%0A%22Coleman+Trapp%22%0A%22Colette+Lush%22%0A%22Colic+Relief%22%0A%22Colin%22%0A%22Colin+Donnell%22%0A%22Colin+Mawby%22%0A%22Collabro%22%0A%22Collective+Soul%22%0A%22College%22%0A%22Collide%22%0A%22Collie+Buddz%22%0A%22Collin+Raye%22%0A%22Collins%22%0A%22Colo+de+Deus%22%0A%22Cologne+Chamber+Orchestra%22%0A%22Colombianas+Salsa+All+Star%22%0A%22Colombo%22%0A%22Colone%22%0A%22Colonel+Abrams%22%0A%22Colonel+Bagshot%22%0A%22Colonel+Reyel%22%0A%22Colorstone%22%0A%22Colour+Castle%22%0A%22Colours+in+Context%22%0A%22Columbine%22%0A%22Colyn%22%0A%22Coma%22%0A%22Comaduster%22%0A%22Combichrist%22%0A%22Comedian+Bob+Marley%22%0A%22Comisario+Pantera%22%0A%22Comma+Dee%

🔎 Batches SPARQL:  20%|█▉        | 120/604 [03:09<12:02,  1.49s/it]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%0A++++SELECT+%3FartistLabel+%3Fbirth+%3Fdeath+%3FcountryLabel+%3FtypeLabel+%3FgenreLabel+WHERE+%7B%0A++++++VALUES+%3FartistLabel+%7B+%22Dahl%22%0A%22Daichi+Miura%22%0A%22Daigo+Hanada%22%0A%22Dailey%22%0A%22Dailucia%22%0A%22Dailyn+Curbelo%22%0A%22Daiquiri%22%0A%22Daisuke+Tanabe%22%0A%22Daisy+Duck%22%0A%22Daisy+Guttridge%22%0A%22Daisy+the+Great%22%0A%22DaisyBanaisy%22%0A%22Dajae%22%0A%22Dakota%22%0A%22Daks%22%0A%22Dale%22%0A%22Dale+Ann+Bradley%22%0A%22Dale+Hawkins%22%0A%22Dale+Howard%22%0A%22Daleema%22%0A%22Daler+Mehndi%22%0A%22Dalex%22%0A%22Dallas%22%0A%22Dallas+Austin%22%0A%22Dallas+Stars%22%0A%22Dallas+Wayne%22%0A%22DallasK%22%0A%22Dallass%22%0A%22Dallerium%22%0A%22Dallomo%22%0A%22Dalmata%22%0A%22Dalora%22%0A%22Dalsin%22%0A%22Dalto%22%0A%22Dalua%22%0A%22Dalva+De+Oliveira%22%0A%22Dalvan%22%0A%22Dam+Swindle%22%0A%22Damage%22%0A%22Damage+Musiq%22%0A%22Damaged+Goods+%28UK%29%22%0A%22Damagepl

🔎 Batches SPARQL:  21%|██▏       | 129/604 [03:24<12:31,  1.58s/it]


KeyboardInterrupt: 

In [7]:
import pandas as pd
import time
from tqdm import tqdm
from SPARQLWrapper import SPARQLWrapper, JSON

# Configuración inicial
archivo_entrada = "../data/artists.csv"
archivo_salida = "wikidata_artists.csv"
tamaño_lote = 10
espera_segundos = 2.5

# Leer archivo de artistas y limpiar nombres
df = pd.read_csv(archivo_entrada, header=None, names=["artist_raw"])

def limpiar_nombre(artista):
    if pd.isna(artista):
        return None
    artista = artista.replace('"', '')
    artista = artista.split(" Featuring ")[0]
    artista = artista.split(" & ")[0]
    artista = artista.split(",")[0]
    return artista.strip()

df["artist"] = df["artist_raw"].apply(limpiar_nombre)
unique_artists = df["artist"].dropna().drop_duplicates().reset_index(drop=True)

# Configurar SPARQL con User-Agent
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="WikidataBatchScript/1.0 (youremail@example.com)")
sparql.setReturnFormat(JSON)

# Función para crear la consulta SPARQL por lote
def crear_consulta_lote(nombres):
    valores = "\n".join(f'"{nombre}"@en' for nombre in nombres)
    return f"""
    SELECT ?artist ?artistLabel ?genreLabel ?countryLabel ?typeLabel ?active ?birthYear ?deathYear WHERE {{
      VALUES ?artistLabel {{ {valores} }}
      ?artist rdfs:label ?artistLabel .
      OPTIONAL {{ ?artist wdt:P136 ?genre . }}
      OPTIONAL {{ ?artist wdt:P27 ?country . }}
      OPTIONAL {{ ?artist wdt:P31 ?type . }}
      OPTIONAL {{ ?artist wdt:P2031 ?active . }}
      OPTIONAL {{ ?artist wdt:P569 ?birthDate .
                  BIND(YEAR(?birthDate) AS ?birthYear) }}
      OPTIONAL {{ ?artist wdt:P570 ?deathDate .
                  BIND(YEAR(?deathDate) AS ?deathYear) }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """

# Proceso por lotes
resultados = []

print(f"🔍 Consultando {len(unique_artists)} artistas en lotes de {tamaño_lote}...\n")

for i in tqdm(range(0, len(unique_artists), tamaño_lote), desc="Batches SPARQL"):
    batch = unique_artists[i:i + tamaño_lote]
    consulta = crear_consulta_lote(batch.tolist())

    try:
        sparql.setQuery(consulta)
        time.sleep(espera_segundos)  # Evitar bloqueo
        respuesta = sparql.query().convert()

        for item in respuesta["results"]["bindings"]:
            resultados.append({
                "artist": item.get("artistLabel", {}).get("value", ""),
                "genre": item.get("genreLabel", {}).get("value", ""),
                "country": item.get("countryLabel", {}).get("value", ""),
                "type": item.get("typeLabel", {}).get("value", ""),
                "active_since": item.get("active", {}).get("value", ""),
                "birth_year": item.get("birthYear", {}).get("value", ""),
                "death_year": item.get("deathYear", {}).get("value", "")
            })

    except Exception as e:
        print(f"❌ Error en SPARQL: {e}")

# Guardar CSV
df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv(archivo_salida, index=False)
print(f"✅ Archivo '{archivo_salida}' creado correctamente.")


🔍 Consultando 2110 artistas en lotes de 10...



Batches SPARQL: 100%|██████████| 211/211 [10:59<00:00,  3.12s/it]

✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [20]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# =====================
# 1. CARGAR Y LIMPIAR
# =====================
print("🔄 Cargando y limpiando datos...")

# Cargar sin cabecera, nombrar la columna como 'raw'
df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

# Separar y limpiar artistas
def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    artistas_limpios = [p.strip().strip('"').strip("'") for p in partes if p.strip()]
    return artistas_limpios

# Expandir y limpiar todos los artistas
todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

# Quitar duplicados
artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))

print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# === ANÁLISIS DE DUPLICADOS ===
print("\n🔍 Análisis de duplicados:")
print("➡️ Total registros originales:", len(df))
print("➡️ Duplicados exactos (filas completas):", df.duplicated().sum())
print("➡️ Duplicados en la columna 'raw':", df["raw"].duplicated().sum())
print("➡️ Después de quitar duplicados:", df["raw"].drop_duplicates().shape[0])
print("➡️ Artistas únicos luego de limpiar:", len(artistas_unicos))

# =====================
# 2. FUNCIONES SPARQL
# =====================

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel WHERE {{
      VALUES ?artistLabel {{ {values} }}
      
      ?artist rdfs:label ?artistLabel.
      FILTER(LANG(?artistLabel) = "en")
      
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.get(WIKIDATA_ENDPOINT, params={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =====================
# 3. PROCESAR BATCHES
# =====================
print("\n🚀 Consultando Wikidata...")

batch_size = 20
results = []

for i in tqdm(range(0, len(artistas_unicos), batch_size), desc="🔎 Batches SPARQL"):
    batch = artistas_unicos[i:i+batch_size]
    data = obtener_datos_wikidata(batch)
    if data:
        for row in data["results"]["bindings"]:
            results.append({
                "artist": row.get("artistLabel", {}).get("value", ""),
                "country": row.get("countryLabel", {}).get("value", ""),
                "type": row.get("typeLabel", {}).get("value", ""),
                "genre": row.get("genreLabel", {}).get("value", ""),
                "birth": row.get("birth", {}).get("value", ""),
                "death": row.get("death", {}).get("value", "")
            })
    time.sleep(0.8)  # respetar política de uso

# =====================
# 4. GUARDAR RESULTADO
# =====================
df_result = pd.DataFrame(results)

# Inferir si está activo o no
df_result["is_active"] = df_result["death"].apply(lambda x: "No" if x else "Yes")

df_result.to_csv("wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")


🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30171

🔍 Análisis de duplicados:
➡️ Total registros originales: 81343
➡️ Duplicados exactos (filas completas): 49906
➡️ Duplicados en la columna 'raw': 49906
➡️ Después de quitar duplicados: 31437
➡️ Artistas únicos luego de limpiar: 30171

🚀 Consultando Wikidata...


🔎 Batches SPARQL:   6%|▌         | 89/1509 [01:36<26:41,  1.13s/it]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%0A++++SELECT+%3FartistLabel+%3Fbirth+%3Fdeath+%3FcountryLabel+%3FtypeLabel+%3FgenreLabel+WHERE+%7B%0A++++++VALUES+%3FartistLabel+%7B+%22Antonio+Cartagena%22%0A%22Antonio+Cirilo%22%0A%22Antonio+Garcia+Isaac%22%0A%22Antonio+Jes%C3%BAs+Asi%C3%A1in+Sanz%22%0A%22Antonio+Jos%C3%A9%22%0A%22Antonio+Marcen%C3%B2%22%0A%22Antonio+Orozco%22%0A%22Antonio+Pappano%22%0A%22Antonio+Rayo+%22Rayito%22%0A%22Antonio+Sol%22%0A%22Antonio+Tarrag%C3%B3+Ros%22%0A%22Antonio+Vega%22%0A%22Antonio+Vivaldi%22%0A%22Antonio+y+Joel+Con+Labranza%22%0A%22Antonis+Kalogiannis%22%0A%22Antony%22%0A%22Antony+Genn%22%0A%22Anton%C3%ADn+Dvo%C5%99%C3%A1k%22%0A%22Antra+Mitra%22%0A%22Antti+L.j.+P%C3%A4%C3%A4kk%C3%B6nen%22+%7D%0A%0A++++++%3Fartist+rdfs%3Alabel+%3FartistLabel.%0A++++++FILTER%28LANG%28%3FartistLabel%29+%3D+%22en%22%29%0A%0A++++++OPTIONAL+%7B+%3Fartist+wdt%3AP27+%3Fcountry.+%7D%0A++++++OPTIONAL+%7B+%3Fartist+wdt%3AP31+%3F

🔎 Batches SPARQL:  11%|█▏        | 171/1509 [03:04<23:54,  1.07s/it]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%0A++++SELECT+%3FartistLabel+%3Fbirth+%3Fdeath+%3FcountryLabel+%3FtypeLabel+%3FgenreLabel+WHERE+%7B%0A++++++VALUES+%3FartistLabel+%7B+%22Bo+Kaspers+Orkester%22%0A%22Bo+Sundstr%C3%B6m%22%0A%22Boards+of+Canada%22%0A%22Boaz+Daniel%22%0A%22Boaz+van+de+Beatz%22%0A%22Bob%22%0A%22Bob+Dylan%22%0A%22Bob+Gunton%22%0A%22Bob+Luman%22%0A%22Bob+Marley%22%0A%22Bob+Moses%22%0A%22Bob+Mould%22%0A%22Bob+Saget%22%0A%22Bob+Seger%22%0A%22Bob+Sinclar%22%0A%22Bob+Smith%22%0A%22Bob+Stevens%22%0A%22Bob+Weir%22%0A%22Bobbi+White%22%0A%22Bobby+%22Blue%22+Bland%22+%7D%0A%0A++++++%3Fartist+rdfs%3Alabel+%3FartistLabel.%0A++++++FILTER%28LANG%28%3FartistLabel%29+%3D+%22en%22%29%0A%0A++++++OPTIONAL+%7B+%3Fartist+wdt%3AP27+%3Fcountry.+%7D%0A++++++OPTIONAL+%7B+%3Fartist+wdt%3AP31+%3Ftype.+%7D%0A++++++OPTIONAL+%7B+%3Fartist+wdt%3AP136+%3Fgenre.+%7D%0A++++++OPTIONAL+%7B+%3Fartist+wdt%3AP569+%3Fbirth.+%7D%0A++++++OPTIONAL+%7B+%3

🔎 Batches SPARQL:  12%|█▏        | 177/1509 [03:11<23:59,  1.08s/it]


KeyboardInterrupt: 

In [26]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. FUNCIONES SPARQL
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel ?awardLabel ?labelLabel WHERE {{
      VALUES ?artistLabel {{ {values} }}
      ?artist rdfs:label ?artistLabel.
      FILTER(LANG(?artistLabel) = "en")
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P166 ?award. }}
      OPTIONAL {{ ?artist wdt:P264 ?label. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 80
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", ""),
                        "label": row.get("labelLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", "")
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
df_result = pd.DataFrame(results)

# Normalizar columnas esperadas
for col in ["artist", "country", "type", "genre", "award", "label", "birth", "death"]:
    if col not in df_result.columns:
        df_result[col] = ""



df_result.to_csv("wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")



🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 30160it [16:21, 30.73it/s]                           


✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. FUNCIONES SPARQL
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel ?awardLabel WHERE {{
      VALUES ?artistLabel {{ {values} }}
      ?artist rdfs:label ?artistLabel.
      FILTER(LANG(?artistLabel) = "en")
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P166 ?award. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers, timeout=20)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", "")
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
df_result = pd.DataFrame(results)

# Ordenar columnas de forma explícita
columnas_ordenadas = ["artist", "country", "type", "genre", "award", "birth", "death"]
df_result = df_result[columnas_ordenadas]

# Guardar CSV
df_result.to_csv("wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente con columnas ordenadas.")




🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL:  86%|████████▌ | 25920/30149 [12:53<01:57, 35.89it/s]

❌ Error en SPARQL: 503 Server Error: Backend fetch failed for url: https://query.wikidata.org/sparql


🔎 Batches SPARQL:  89%|████████▊ | 26730/30149 [13:22<01:51, 30.73it/s]

❌ Error en SPARQL: 503 Server Error: Backend fetch failed for url: https://query.wikidata.org/sparql


🔎 Batches SPARQL: 30180it [15:11, 33.12it/s]                           


✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. FUNCIONES SPARQL
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      FILTER(LANG(?name) = "en")
      
      # Filtrar por instancias válidas (solo humanos o grupos musicales)
      ?artist wdt:P31 ?type.
      FILTER(?type IN (wd:Q5, wd:Q215380, wd:Q2088357))
      
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    return query


def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", "")
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
df_result = pd.DataFrame(results)

# Reordenar columnas correctamente
df_result = df_result[["artist", "country", "type", "genre", "award", "birth", "death"]]
df_result.to_csv("../data/wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")


🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 30180it [14:12, 35.40it/s]                           

✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. CONSULTA SPARQL FILTRADA
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
TIPOS_VALIDOS = ["human", "musical group", "vocal group", "music ensemble", "band"]

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    tipos_validos = '", "'.join(TIPOS_VALIDOS)
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      FILTER(LANG(?name) = "en")
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P166 ?award. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
      FILTER(?typeLabel IN ("{tipos_validos}"))
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "gender": row.get("genderLabel", {}).get("value", "")  # Nuevo campo
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "type", "genre", "award", "birth", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)
df_result.to_csv("../data/wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")

🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL:  34%|███▍      | 10260/30149 [04:40<08:13, 40.34it/s]

❌ Error en SPARQL: HTTPSConnectionPool(host='query.wikidata.org', port=443): Max retries exceeded with url: /sparql (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DF407D53D0>, 'Connection to query.wikidata.org timed out. (connect timeout=None)'))


🔎 Batches SPARQL:  54%|█████▍    | 16230/30149 [07:13<06:11, 37.43it/s]


KeyboardInterrupt: 

In [4]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. CONSULTA SPARQL CON TODOS LOS CAMPOS
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel ?awardLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}      
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "")  # Nuevo campo  # Si no hay premios, asignamos "No awards"
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "type", "genre", "award", "birth", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)
df_result.to_csv("../data/wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")


🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 30180it [26:21, 19.08it/s]                           


✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [5]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. CONSULTA SPARQL CON TODOS LOS CAMPOS
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}  # Género con P21
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")  # Valor por defecto si no hay género
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "type", "genre", "award", "birth", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)
df_result.to_csv("../data/wikidata_artists.csv", index=False)
print("✅ Archivo 'wikidata_artists.csv' creado correctamente.")


🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 30180it [29:06, 17.28it/s]                            


✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [ ]:
# Workshop_2/source/extract/extract_api.py

import pandas as pd
import requests
import time
import csv
from tqdm import tqdm

import pandas as pd

# Cargar archivo original
df = pd.read_csv("../data/artists.csv", header=None, names=["artist_raw"])

# Limpieza básica
def limpiar_nombre(artista):
    if pd.isna(artista):
        return None
    artista = artista.replace('"', '')
    artista = artista.split(" Featuring ")[0]
    artista = artista.split(" & ")[0]
    artista = artista.split(",")[0]
    return artista.strip()

df["artist"] = df["artist_raw"].apply(limpiar_nombre)

# Eliminar duplicados y valores vacíos
unique_artists = df["artist"].dropna().drop_duplicates().reset_index(drop=True)

# Lista para almacenar los resultados
results = []


import pandas as pd
import requests
import time
from tqdm import tqdm

API_KEY = '1'  # Puedes reemplazar con tu propia API Key si tienes una
INPUT_FILE = 'Workshop_2/data/artists.csv'
OUTPUT_FILE = 'Workshop_2/data/api_data.csv'

def get_artist_info(artist_name, retries=3, delay=1):
    url = f'https://www.theaudiodb.com/api/v1/json/{API_KEY}/search.php?s={artist_name}'
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data['artists'] is not None:
                    return data['artists'][0]  # Tomamos solo el primero si hay varios
                else:
                    print(f"⚠️ No se encontró información para {artist_name}")
                    return None
            else:
                print(f"⚠️ Error {response.status_code} para {artist_name}")
        except Exception as e:
            print(f"⚠️ Excepción para {artist_name}: {e}. Intento {attempt + 1}/{retries}")
            time.sleep(delay)
    return None

def extract_all_artists():
    df = pd.read_csv(INPUT_FILE)
    artists = df['artists'].dropna().unique()

    records = []
    for artist in tqdm(artists, desc="Procesando artistas"):
        artist_info = get_artist_info(artist)
        if artist_info:
            records.append({
                'artist_name': artist,
                'idArtist': artist_info.get('idArtist'),
                'strCountry': artist_info.get('strCountry'),
                'intBornYear': artist_info.get('intBornYear'),
                'intDiedYear': artist_info.get('intDiedYear'),
                'strGender': artist_info.get('strGender'),
                'strGenre': artist_info.get('strGenre'),
                'strStyle': artist_info.get('strStyle'),
                'strWebsite': artist_info.get('strWebsite'),
                'strBiographyEN': artist_info.get('strBiographyEN'),
            })

    result_df = pd.DataFrame(records)
    result_df.to_csv(OUTPUT_FILE, index=False)
    print(f"✅ Extracción finalizada. Datos guardados en {OUTPUT_FILE}")

if __name__ == "__main__":
    extract_all_artists()




Procesando artistas:   0%|          | 0/2110 [00:00<?, ?it/s]

⚠️ Respuesta vacía para artist. Intento 1/3
⚠️ Respuesta vacía para artist. Intento 2/3
⚠️ Respuesta vacía para artist. Intento 3/3


Procesando artistas:   0%|          | 1/2110 [00:12<7:15:08, 12.38s/it]

⚠️ Respuesta vacía para Billie Eilish. Intento 1/3
⚠️ Respuesta vacía para Billie Eilish. Intento 2/3


Procesando artistas:   0%|          | 2/2110 [00:13<3:22:34,  5.77s/it]

...

Procesando artistas:  18%|█▊        | 376/2110 [07:38<32:17,  1.12s/it]

⚠️ Respuesta vacía para Seattle Symphony. Intento 3/3
⚠️ Respuesta vacía para Karim Sulayman; Jeannette Sorrell. Intento 1/3
⚠️ Respuesta vacía para Karim Sulayman; Jeannette Sorrell. Intento 2/3


Procesando artistas:  18%|█▊        | 376/2110 [07:40<35:21,  1.22s/it]

⚠️ Respuesta vacía para Karim Sulayman; Jeannette Sorrell. Intento 3/3


KeyboardInterrupt: 

In [ ]:

# Consultar MusicBrainz para cada artista
headers = {'User-Agent': 'ETLWorkshop/1.0 (tucorreo@dominio.com)'}
datos_artistas = []

for artist in list(artistas):  # puedes ajustar el número para pruebas
    query = artist.replace(" ", "+")
    url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{query}&fmt=json&limit=1"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data['artists']:
            a = data['artists'][0]
            datos_artistas.append({
                'artist_name': artist,
                'artist_id': a.get('id'),
                'country': a.get('country'),
                'begin_date': a.get('life-span', {}).get('begin'),
                'end_date': a.get('life-span', {}).get('end'),
                'type': a.get('type'),
                'gender': a.get('gender'),
                'disambiguation': a.get('disambiguation'),
            })
    else:
        print(f"❌ Error con {artist}: {response.status_code}")
    
    time.sleep(1)

# Guardar como CSV
df_api = pd.DataFrame(datos_artistas)
df_api.to_csv("../data/musicbrainz_artists.csv", index=False)

print("✅ Datos de MusicBrainz guardados en api_data.csv.")


✅ Datos de MusicBrainz guardados en api_data.csv.


In [3]:
df = pd.read_csv("../data/musicbrainz_artists.csv")
df.head()

,artist_name,artist_id,country,begin_date,end_date,type,gender,disambiguation
0,Rhonda Vincent And The Rage,b6310fcc-30ea-41a9-9cda-1931f569d76f,US,NaN,NaN,Group,NaN,NaN
1,Yolanda Kondonassis,73e9c3b8-ef39-48e5-8b08-f746dcf0c765,US,1963,NaN,Person,female,harpist
2,John Hartford,3178cc4c-cd9f-4842-96fb-28bf9cfec591,US,1937-12-30,2001-06-04,Person,male,NaN
3,Sheryl Crow,80ccfede-c258-4575-a7ad-c982e9932e0f,US,1962-02-11,NaN,Person,female,American singer-songwriter and musician
4,Neal Hefti,75770f14-cbe0-4b06-8825-14797098ff2a,US,1922-10-29,2008-10-11,Person,male,NaN
